In [1]:
import numpy as np
import sympy as sp
from sympy.physics.quantum import TensorProduct, Dagger
from sympy.physics.quantum.trace import Tr
from scipy.optimize import minimize
from IPython.display import display

In [2]:
# Define |H>, |V> in 
H = sp.Matrix([1, 0])
V = sp.Matrix([0, 1])
I2 = sp.eye(2)

# Define Bell state vectors
phi_plus = (TensorProduct(H, H) + TensorProduct(V, V)) / sp.sqrt(2)
phi_minus = (TensorProduct(H, H) - TensorProduct(V, V)) / sp.sqrt(2)
psi_plus = (TensorProduct(H, V) + TensorProduct(V, H)) / sp.sqrt(2)
psi_minus = (TensorProduct(H, V) - TensorProduct(V, H)) / sp.sqrt(2)

# returns the density matrix corresponding to a specified state vector def density_operator_from_vector(state_vector): return TensorProduct(state_vector, Dagger(state_vector))
def density_operator_from_vector(state_vector):
    return TensorProduct(state_vector, Dagger(state_vector))

# returns operator for action of HWP 1 particle state in H,V basis
# note that the angles here are in terms of the angle of the EM field oscillation orientation, not the filter orientation
# the angle for the filter is be half the angle specified here
def half_wave_plate_sympy(theta):
    c = sp.cos(theta)
    s = sp.sin(theta)
    return sp.Matrix([[c, s], [s, -c]])


def find_correlation_function(initial_state_density, lambdify=False):

    # Define sympy variables 
    theta_a, theta_b = sp.symbols('theta_a, theta_b', real=True)

    # Define Operator corresponding to two HWPs
    HWP_operator = sp.simplify(TensorProduct(half_wave_plate_sympy(theta_a), half_wave_plate_sympy(theta_b)))

    # Define coincidence measurement operators
    P_V = V * Dagger(V)  # |V><V| projection operator
    P_H = H * Dagger(H)  # |H><H| projection operator
    VH_operator = TensorProduct(P_V, P_H)
    VV_operator = TensorProduct(P_V, P_V)
    HH_operator = TensorProduct(P_H, P_H)
    HV_operator = TensorProduct(P_H, P_V)

    # Apply HWP operator on initial state density
    state_density = sp.simplify(HWP_operator * initial_state_density * Dagger(HWP_operator))

    # calculate correlation function

    C = Tr(state_density * HH_operator) - Tr(state_density * HV_operator)- Tr(state_density * VH_operator) + Tr(state_density * VV_operator)
    C = C.simplify()

    if lambdify:
        return sp.lambdify([theta_a, theta_b], C)
    else:
        return C 


def find_CHSH_angles(initial_state_density):

    theta_a_0, theta_a_1, theta_b_0, theta_b_1 = sp.symbols('theta_a_0 theta_a_1 theta_b_0 theta_b_1', real=True)

    # get the lambdified correlation function (for faster computation)
    C = find_correlation_function(initial_state_density, lambdify=True)

    # build S function from correlation functions (make in terms of multiples of pi)
    def S(x):
        a0 = x[0] * np.pi 
        a1 = x[1] * np.pi 
        b0 = x[2] * np.pi 
        b1 = x[3] * np.pi 
        return -np.abs(C(a0, b0) + C(a1, b0) + C(a0, b1) - C(a1, b1))

    x0 = [0, 1/4, -1/8, 3/8]
    bounds = [(-1,1),(-1,1),(-1,1),(-1,1)]
    constraint = {'type': 'eq', 'fun': lambda x: x[0]} # force first angle to be 0

    result = minimize(fun=S, x0=x0, bounds=bounds, constraints=constraint)
    maximum = -result["fun"]
    angles = result['x']

    return maximum, angles 

def print_summary(initial_state, initial_state_density, S, CHSH_angles):

    if initial_state is not None:
        print("For the initial state:")
        display(initial_state)

    print("The correlation function has the form:")
    display(find_correlation_function(initial_state_density))

    print("We find the following optimal CHSH angles (in multiples of pi):")
    print(f"a0:\t{CHSH_angles[0]:.4f}, a1:\t{CHSH_angles[1]:.4f}\nb0:\t{CHSH_angles[2]:.4f}, b1:\t{CHSH_angles[3]:.4f}")

    print("\nAnd measurements taken at this angle will produce as CHSH value S of")
    print(f"S = {S}")


In [3]:
initial_state = phi_plus
initial_state_density = density_operator_from_vector(initial_state)
S, CHSH_angles = find_CHSH_angles(initial_state_density)
print_summary(phi_plus, initial_state_density, S,CHSH_angles)


For the initial state:


Matrix([
[sqrt(2)/2],
[        0],
[        0],
[sqrt(2)/2]])

The correlation function has the form:


cos(2*theta_a - 2*theta_b)

We find the following optimal CHSH angles (in multiples of pi):
a0:	0.0000, a1:	0.2500
b0:	-0.3750, b1:	0.3750

And measurements taken at this angle will produce as CHSH value S of
S = 2.8284271142376207


In [4]:
mixed_state_50_50 = 0.5*TensorProduct(TensorProduct(H, H), Dagger(TensorProduct(H,H))) + 0.5*TensorProduct(TensorProduct(V, V), Dagger(TensorProduct(V,V)))

S, CHSH_angles = find_CHSH_angles(mixed_state_50_50)
print_summary(None, mixed_state_50_50, S,CHSH_angles)

The correlation function has the form:


0.5*cos(2*theta_a - 2*theta_b) + 0.5*cos(2*theta_a + 2*theta_b)

We find the following optimal CHSH angles (in multiples of pi):
a0:	0.0000, a1:	1.0000
b0:	1.0000, b1:	1.0000

And measurements taken at this angle will produce as CHSH value S of
S = 2.0
